In [1]:
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import numpy as np
import pickle
tokenizer = AutoTokenizer.from_pretrained("vinai/PhoGPT-4B-Chat")
model = AutoModel.from_pretrained("vinai/PhoGPT-4B-Chat")

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

In [2]:
def get_embedding(text):
    
    inputs = tokenizer(text, return_tensors="pt", truncation= True, padding= True, max_length = 512)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.detach().numpy().mean(1)
    return embedding

In [4]:
import pandas as pd
import os
path_unsafe = "/kaggle/input/unsafety-data/merge.csv"
path_safe = "/kaggle/input/safety/alpaca_gpt4_data_vi_transformed.csv"

df_unsafe = pd.read_csv(path_unsafe)
df_safe = pd.read_csv(path_safe)

if os.path.exists('/kaggle/working/texts_labels.pkl'):
    with open('/kaggle/working/texts_labels.pkl', 'rb') as f:
        texts, labels = pickle.load(f)
    print("Loaded texts and labels from texts_labels.pkl")
else:
    texts = []
    labels = []
    print("File texts_labels.pkl does not exist. You may need to generate it first.")

File texts_labels.pkl does not exist. You may need to generate it first.


In [5]:
start = 0
end = 5
index = 0
for _,row in df_unsafe.iterrows():
    if index < start:
        index += 1
        continue
    texts.append(get_embedding(row.iloc[0]))
    labels.append(0)
    print(index)
    index += 1
    if index == end:
        break

        
start = 0
end = 5
index = 0
for _,row in df_safe.iterrows():
    if index < start:
        index += 1
        continue
    texts.append(get_embedding(row.iloc[0]))
    labels.append(1)
    print(index)
    index += 1
    if index == end:
        break
        


0
1
2
3
4
0
1
2
3
4


In [7]:
with open('/kaggle/working/texts_labels.pkl', 'wb') as f:
    pickle.dump((texts, labels), f)

In [ ]:
X = np.array(texts)
X = X.reshape(X.shape[0],-1)


y = np.array(labels)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape, X.shape, y.shape)
model_logistic = LogisticRegression(max_iter=1000)
model_logistic.fit(X_train, y_train)

y_pred = model_logistic.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

In [ ]:
import joblib

joblib.dump(model_logistic, '/kaggle/working/logistic_regression_model.pkl')
model_logistic = joblib.load('/kaggle/working/logistic_regression_model.pkl')

y_pred = model_logistic.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)